In [1]:
from nltk.tokenize import sent_tokenize
import pandas as pd
import sys
import time
import os



#sys.path.append("../../oats")
#from oats.pubmed.query import search, fetch_details
#from oats.utils.utils import save_to_pickle, load_from_pickle

In [2]:
from Bio import Entrez

def search(query,limit):
    Entrez.email = 'a@b.com'
    handle = Entrez.esearch(db='pubmed',sort='relevance',retmax=limit,retmode='xml',term=query)
    results = Entrez.read(handle)
    return results


def fetch_details(id_list):
    ids = ','.join(id_list)
    Entrez.email = 'a@b.com'
    handle = Entrez.efetch(db='pubmed', retmode='xml', id=ids)
    results = Entrez.read(handle)
    return results

In [3]:
df = pd.read_csv("../../plant-data/genes_texts_annots.csv")


In [14]:
query = "XTC1 AND arabidopsis"

limit = 100
results = search(query, limit)

pubmed_ids = results["IdList"]
print(pubmed_ids)

papers = fetch_details(pubmed_ids)

['9294189']


In [15]:
for paper in papers['PubmedArticle']:
    abstract_text = paper['MedlineCitation']['Article']['Abstract']["AbstractText"][0]
    print(abstract_text)

We describe mutations of three genes in Arabidopsis thaliana-extra cotyledon1 (xtc1), extra cotyledon2 (xtc2), and altered meristem programming1 (amp1)-that transform leaves into cotyledons. In all three of these mutations, this transformation is associated with a change in the timing of events in embryogenesis. xtc1 and xtc2 delay the morphogenesis of the embryo proper at the globular-to-heart transition but permit the shoot apex to develop to an unusually advanced stage late in embryogenesis. Both mutations have little or no effect on seed maturation and do not affect the viability of the shoot or the rate of leaf initiation after germination. amp1 perturbs the pattern of cell division at an early globular stage, dramatically increases the size of the shoot apex and, like xtc1 and xtc2, produces enlarged leaf primordia during seed development. These unusual phenotypes suggest that these genes play important regulatory roles in embryogenesis and demonstrate that the development of the

In [8]:
import requests
#from unidecode import unidecode


Bioconcept = "Gene"

InputSTR = '9294189'


# submit request
r = requests.post("https://www.ncbi.nlm.nih.gov/research/pubtator-api/annotations/annotate/submit/"+Bioconcept , data = InputSTR.encode('utf-8'))
if r.status_code != 200 :
    print ("[Error]: HTTP code "+ str(r.status_code))
else:
    SessionNumber = r.text
    print ("Thanks for your submission. The session number is : "+ SessionNumber + "\n")
    #outputfile.write(SessionNumber+"\t"+filename+"\n")

Thanks for your submission. The session number is : 4680-5952-6882-3537



In [12]:
print(r.url)

https://www.ncbi.nlm.nih.gov/research/pubtator-api/annotations/annotate/submit/Gene


In [ ]:
papers = fetch_details(pubmed_ids)
for paper in papers["PubmedArticle"]

In [ ]:
# Getting a sampling of text that is from some domain of interest.
interesting_texts = []
limit = 1000000
fetch_batch_size = 1000
seconds_between_fetch = 5

query = "maize AND phenotype"
print("querying")
results = search(query, limit)
id_list = results['IdList']
print("{} abstracts found with this query".format(len(id_list)))
if len(id_list) > 0:
	print("fetching details")
	id_list_batches = [id_list[x:x+fetch_batch_size] for x in range(0, len(id_list), fetch_batch_size)]
	print("splitting into {} batches".format(len(id_list_batches)))
	for i,id_list_batch in enumerate(id_list_batches):
		print("batch {}".format(i))
		papers = fetch_details(id_list_batch)
		for i, paper in enumerate(papers['PubmedArticle']):
			try:
				abstract_text = paper['MedlineCitation']['Article']['Abstract']["AbstractText"][0]
				interesting_texts.append(abstract_text)
			except KeyError:
				continue
		time.sleep(seconds_between_fetch)
interesting_text = " ".join(interesting_texts)

In [ ]:
# Creating the dataset of sentences related to Arabidopsis phenotypes.
path = "../data/corpus_related_files/brat_annotations_ath_corpus"
query = "arabidopsis AND phenotype"
limit = 100
results = search(query, limit)
id_list = results["IdList"]
papers = fetch_details(id_list)

# A list of the sentences and a list of the corresponding PubMed IDs of the same length.
sentences = []
pubmed_ids = []
for i, paper in enumerate(papers['PubmedArticle']):
    try:
        abstract_text = paper['MedlineCitation']['Article']['Abstract']["AbstractText"][0]
        abstract_sentences = sent_tokenize(abstract_text)
        for sentence in abstract_sentences:
            sentences.append(sentence)
            pubmed_ids.append(str(id_list[i]))
    except KeyError